In [20]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

CONFIG = {
    "SEED": 42,
    "IMG_SIZE": 128,
    "BATCH_SIZE": 32,
    "EPOCHS": 20,
    "LR": 0.001,
    "DEVICE": 'cuda' if torch.cuda.is_available() else 'cpu',
    "NUM_CLASSES": 18
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(CONFIG["SEED"])

In [21]:
class OnePieceDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, mode='train'):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        if self.mode in ['train', 'val']:
            rel_path = row['image_path'].replace('\\', '/')
            img_path = os.path.join(self.root_dir, rel_path)
            label = int(row['label'])
        else:
            img_id = row['id']
            img_path = os.path.join(self.root_dir, 'test', f"{img_id}") 
            possible_exts = ['.jpg', '.png', '.jpeg']
            for ext in possible_exts:
                temp_path = os.path.join(self.root_dir, 'test', f"{img_id}{ext}")
                if os.path.exists(temp_path):
                    img_path = temp_path
                    break
            label = -1

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            image = Image.new('RGB', (CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"]))

        if self.transform:
            image = self.transform(image)

        if self.mode == 'test':
            return image, row['id']
        else:
            return image, torch.tensor(label, dtype=torch.long)

In [22]:
train_transforms = transforms.Compose([
    transforms.Resize((CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"])),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((CONFIG["IMG_SIZE"], CONFIG["IMG_SIZE"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

csv_root = 'data'               # Где лежат .csv
img_root = 'data/splitted'      # Где лежат папки train и test

train_df_full = pd.read_csv(os.path.join(csv_root, 'train_annotations.csv'))

train_df, val_df = train_test_split(
    train_df_full, 
    test_size=0.2, 
    random_state=CONFIG["SEED"], 
    stratify=train_df_full['label']
)

train_dataset = OnePieceDataset(train_df, img_root, transform=train_transforms, mode='train')
val_dataset = OnePieceDataset(val_df, img_root, transform=val_transforms, mode='val')

train_loader = DataLoader(train_dataset, batch_size=CONFIG["BATCH_SIZE"], shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=CONFIG["BATCH_SIZE"], shuffle=False, num_workers=0)

In [23]:
class CompactCNN(nn.Module):
    def __init__(self, num_classes=18):
        super(CompactCNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        
        self.global_pool = nn.AdaptiveAvgPool2d(1) 
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = CompactCNN(num_classes=CONFIG["NUM_CLASSES"]).to(CONFIG["DEVICE"])

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=CONFIG["LR"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    for images, labels in tqdm(loader, desc="Train"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
    return running_loss / len(loader), f1_score(all_labels, all_preds, average='macro')

def val_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Val"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    return running_loss / len(loader), f1_score(all_labels, all_preds, average='macro')

In [25]:
best_f1 = 0.0
best_model_path = "model_compact.pth"

for epoch in range(CONFIG["EPOCHS"]):
    print(f"Epoch {epoch+1}/{CONFIG['EPOCHS']}")
    
    train_loss, train_f1 = train_epoch(model, train_loader, optimizer, criterion, CONFIG["DEVICE"])
    val_loss, val_f1 = val_epoch(model, val_loader, criterion, CONFIG["DEVICE"])
    
    scheduler.step(val_f1)
    
    print(f"Train Loss: {train_loss:.4f} | F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f} | F1: {val_f1:.4f}")
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), best_model_path)

print(f"Best Val F1: {best_f1}")
print(f"Model Size: {os.path.getsize(best_model_path)/1024/1024:.2f} MB")

Epoch 1/20


Train:   4%|▍         | 3/73 [00:01<00:45,  1.55it/s]/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Val: 100%|██████████| 19/19 [00:06<00:00,  2.86it/s]


Train Loss: 2.6013 | F1: 0.1810
Val Loss: 2.1392 | F1: 0.2884
Epoch 2/20


Train:   8%|▊         | 6/73 [00:03<00:41,  1.62it/s]/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Val: 100%|██████████| 19/19 [00:06<00:00,  2.87it/s]


Train Loss: 2.2450 | F1: 0.2973
Val Loss: 1.9340 | F1: 0.3662
Epoch 3/20


...

Train Loss: 0.8069 | F1: 0.7586
Val Loss: 0.8421 | F1: 0.7468
Epoch 19/20


Train:   0%|          | 0/73 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Val: 100%|██████████| 19/19 [00:08<00:00,  2.34it/s]


Train Loss: 0.7946 | F1: 0.7692
Val Loss: 0.8453 | F1: 0.7585
Epoch 20/20


Train:   8%|▊         | 6/73 [00:05<00:56,  1.18it/s]/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Val: 100%|██████████| 19/19 [00:07<00:00,  2.63it/s]

Train Loss: 0.7578 | F1: 0.7885
Val Loss: 1.0040 | F1: 0.7002
Best Val F1: 0.7584740947885336
Model Size: 3.77 MB


In [26]:
model.load_state_dict(torch.load(best_model_path))
model.eval()

sample_sub = pd.read_csv(os.path.join(csv_root, 'submission.csv'))
test_dataset = OnePieceDataset(sample_sub, img_root, transform=val_transforms, mode='test')
test_loader = DataLoader(test_dataset, batch_size=CONFIG["BATCH_SIZE"], shuffle=False, num_workers=0)

submission_data = []

with torch.no_grad():
    for images, ids in tqdm(test_loader, desc="Inference"):
        images = images.to(CONFIG["DEVICE"])
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        preds = preds.cpu().numpy()
        for img_id, label in zip(ids, preds):
            submission_data.append({'id': img_id, 'label': label})

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission_final.csv', index=False)
submission_df.head()

/var/folders/kn/k7fnf3gj53sbht00y8v6k4900000gn/T/ipykernel_4902/4045760855.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_pa

,id,label
0,c41628b1-4781-4392-ac8d-6bfe981f73f9,10
1,f114acb3-fe18-478b-a19a-1f4cbe098851,7
2,d952ecfe-750c-44b2-96c2-1cac1a4ee146,2
3,2c14ec77-44ca-4b3c-b470-96286411c617,14
4,712c3ce9-750a-4cc4-8f94-f8033c31cb2c,0
